In [2]:
import tensorflow as tf
import os

In [3]:
data_dir = '../../../Other/datasets/cats_vs_dogs'

train_cats_dir = data_dir + '/train/cats/'
train_dogs_dir = data_dir + '/train/dogs/'
train_tfrecord_file = data_dir + '/train/train.tfrecords'

#### 写入

In [ ]:
train_cat_filenames = [train_cats_dir + filename for filename in os.listdir(train_cats_dir)]
train_dog_filenames = [train_dogs_dir + filename for filename in os.listdir(train_dogs_dir)]
train_filenames = train_cat_filenames + train_dog_filenames

train_labels = [0] * len(train_cat_filenames) + [1] * len(train_dog_filenames)

In [11]:
with tf.io.TFRecordWriter(train_tfrecord_file) as writer:
    for filename, label in zip(train_filenames, train_labels):
        # 1,数据img,label
        image = open(filename, 'rb').read()

        # 2,Features字典
        feature = {
            'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),  # image的数据类型为bytes
            'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))  # label的数据类型为int
        }

        # 3,Features字典转换为Example对象并序列化
        example = tf.train.Example(features=tf.train.Features(feature=feature)).SerializeToString()

        # 4,写入到TFRecord文件
        writer.write(example)

#### 加载

In [16]:
# 1. 读取TFRecore文件
train_dataset = tf.data.TFRecordDataset(train_tfrecord_file)

# 2,定义Feature结构,告诉解码器每个Feature的类型是什么
feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64),
}


def _parse_example(example_string):
    # 3,将TFRecord文件中的每一个序列化的tf.train.Example解码
    feature_dict = tf.io.parse_single_example(example_string, feature_description)

    feature_dict['image'] = tf.io.decode_jpeg(feature_dict['image'])  # 解码JPEG图片
    feature_dict['image'] = tf.image.resize(feature_dict['image'], [256, 256]) / 255.0
    return feature_dict['image'], feature_dict['label']

In [17]:
# 数据处理
train_dataset = train_dataset.map(_parse_example)
train_dataset = train_dataset.shuffle(buffer_size=23000)
train_dataset = train_dataset.batch(16)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [18]:
for i in train_dataset.take(1):
    print(i)

(<tf.Tensor: shape=(16, 256, 256, 3), dtype=float32, numpy=
array([[[[4.16541964e-01, 2.32228234e-01, 1.61640003e-01],
         [4.32866722e-01, 2.48552993e-01, 1.77964747e-01],
         [4.49149817e-01, 2.64836103e-01, 1.94370404e-01],
         ...,
         [4.74664539e-01, 4.19640034e-01, 4.08059150e-01],
         [4.01318610e-01, 3.47948492e-01, 3.31588209e-01],
         [3.93928319e-01, 3.42947930e-01, 3.19418520e-01]],

        [[3.56403798e-01, 1.72090054e-01, 1.02099247e-01],
         [3.63706112e-01, 1.79392383e-01, 1.09401569e-01],
         [3.73535395e-01, 1.89221665e-01, 1.19344078e-01],
         ...,
         [4.58246082e-01, 4.02927548e-01, 3.91346663e-01],
         [4.11766499e-01, 3.57980996e-01, 3.41970772e-01],
         [4.02410895e-01, 3.50833058e-01, 3.28199804e-01]],

        [[3.46652865e-01, 1.61942422e-01, 9.99907851e-02],
         [3.46652865e-01, 1.62339151e-01, 9.95940566e-02],
         [3.42654467e-01, 1.58340752e-01, 9.55956578e-02],
         ...,
         